In [6]:
from models.external.NAFNet.NAFNet_arch import NAFNet
import torch
import numpy as np

from common.utils import save_reconstructions
from pathlib import Path
from collections import defaultdict
import fastmri
import h5py
import os
from torch.utils.data import DataLoader, Dataset
from typing import Optional, Tuple, List

PUBLIC_ACCS = ["acc4", "acc5", "acc8"]
root_path = "/home/Data/leaderboard"
out_path = "reconstructions/grappa"

In [ ]:
for acc in os.listdir(root_path):
    reconstructions = defaultdict(dict)
    file_list = os.listdir(f"{root_path}/{acc}/image")
    for file in file_list:
        grappa = h5py.File(f"{root_path}/{acc}/image/{file}", "r")["image_grappa"]
        for i in range(grappa.shape[0]):
            reconstructions[file][i] = np.array(grappa[i])
    for fname in reconstructions:
        reconstructions[fname] = np.stack([reconstructions[fname][slice] for slice in sorted(reconstructions[fname])])

    if acc in PUBLIC_ACCS:
        save_reconstructions(reconstructions, Path(f"{out_path}/public"))

    else:
        save_reconstructions(reconstructions, Path(f"{out_path}/private"))

In [7]:
Grappa_DataType = Tuple[
    torch.Tensor, # Grappa Image
    Optional[torch.Tensor], # Ground Truth Image (target)
]

class GrappaDataset(Dataset):
    def __init__(self, root_path: str, acc_list: List[str]) -> None:
        self.root_path = root_path
        for acc in acc_list:
            self.file_list = os.listdir(f"{root_path}/{acc}/image")
            self.grappa_data = {}
            for file in self.file_list:
                grappa = h5py.File(f"{root_path}/{acc}/image/{file}", "r")["image_grappa"]
                for i in range(grappa.shape[0]):
                    self.grappa_data[file][i] = torch.tensor(grappa[i])
        self.file_list = os.listdir(f"{root_path}/{acc}/image")
        self.grappa_data = {}
        for file in self.file_list:
            grappa = h5py.File(f"{root_path}/{acc}/image/{file}", "r")["image_grappa"]
            for i in range(grappa.shape[0]):
                self.grappa_data[file][i] = torch.tensor(grappa[i])

    def __len__(self) -> int:
        return len(self.file_list)

    def __getitem__(self, idx: int) -> Grappa_DataType:
        return self.grappa_data[self.file_list[idx]]

grappa_dataloaders = {}
for acc in os.listdir(root_path):
    grappa_dataset = GrappaDataset(root_path, acc)
    grappa_raw_data = h5py.File(f"{root_path}/{acc}/image/{file}", "r")["image_grappa"]
    grappa_data = torch.tensor(grappa_raw_data)

NameError: name 'file' is not defined

In [ ]:
model = NAFNet(
    
)